In [20]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:9 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists

In [21]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-12 17:06:16--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.4’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2023-03-12 17:06:17 (10.5 MB/s) - ‘postgresql-42.2.9.jar.4’ saved [914037/914037]



In [22]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


# Extract the Amazon Data into Spark DataFrame

In [23]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

# Load file
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz" #enter correct address here
spark.sparkContext.addFile(url)
grocery_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", header=True)

grocery_df.show(20)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [24]:
# Get the number of rows in the DataFrame.
grocery_df.count()


2402458

# Transform the Data

## Create the "review_id_table".

In [25]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = grocery_df.select('review_id', 'customer_id', 'product_id', 'product_parent', 'review_date')
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R26MV8D0KG6QI6|   42521656|B000SAQCWC|     159713740| 2015-08-31|
|R1OF8GP57AQ1A0|   12049833|B00509LVIQ|     138680402| 2015-08-31|
|R3VDC1QB6MC4ZZ|     107642|B00KHXESLC|     252021703| 2015-08-31|
|R12FA3DCF8F9ER|    6042304|B000F8JIIC|     752728342| 2015-08-31|
| RTWHVNV6X4CNJ|   18123821|B004ZWR9RQ|     552138758| 2015-08-31|
| RIG9AWFOGRDVO|   23649464|B00AL6QBZ6|     681475449| 2015-08-31|
|R1S1XSG4ZCHDGS|   32778285|B00BCNSTRA|     578681693| 2015-08-31|
| RB15NBVY5ELVW|   46612941|B0089MM2BG|     350968436| 2015-08-31|
| R56358YM1ZJ7I|   31525399|B00Y1C9770|     729982780| 2015-08-31|
|R1ODXB3C9UP3NL|   19624355|B00J074W94|       2499702| 2015-08-31|
|R155ZMVLD5C9BP|   31910375|B0001VKKOO|     189938205| 2015-08-31|
|R2IXW43PUNYM0G|   12816533|B007TGH4CK|     398128262| 2015-08

In [26]:
# Check datatypes
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: string (nullable = true)



In [27]:
# Change datatypes to match schema
from pyspark.sql.types import StringType, DateType, IntegerType

review_id_df = review_id_df \
  .withColumn("customer_id" ,
              review_id_df["customer_id"]
              .cast(IntegerType()))   \
  .withColumn("product_parent",
              review_id_df["product_parent"]
              .cast(IntegerType()))    \
  .withColumn("review_date"  ,
              review_id_df["review_date"]
              .cast(DateType()))

# Recheck datatypes
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



## Create the "products" Table

In [28]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = grocery_df.select('product_id', 'product_title')
products_df.show()
products_df.count()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000SAQCWC|The Cravings Plac...|
|B00509LVIQ|Mauna Loa Macadam...|
|B00KHXESLC|Organic Matcha Gr...|
|B000F8JIIC|15oz Raspberry Ly...|
|B004ZWR9RQ|Stride Spark Kine...|
|B00AL6QBZ6|Herr's Popcorn Ho...|
|B00BCNSTRA|Larabar uber, 1.4...|
|B0089MM2BG|Shirakiku Soba No...|
|B00Y1C9770|Jif Chocolate Nut...|
|B00J074W94|Orgain Organic Pl...|
|B0001VKKOO|Bragg - All Natur...|
|B007TGH4CK|Wholesome Sweeten...|
|B002HMN6SC|Kadoya Pure Sesam...|
|B004NRHAZO|Nishiki Premium B...|
|B00V7LJIG8|Everly Passion Fr...|
|B0000E2YFI|Charms Blue Razzb...|
|B00VK5SQOQ|Food Should Taste...|
|B00XDXMLL2|Skippy Creamy Pea...|
|B00MN4KX3A|Celestial Seasoni...|
|B004NTCE1M|Nutiva Organic Vi...|
+----------+--------------------+
only showing top 20 rows



2402458

In [29]:
# Drop duplicates and get new row count to ensure
products_df = products_df.dropDuplicates()
products_df.count()


305538

In [30]:
# Check that datatypes match schema
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



## Create the "customers" Table

---



In [31]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 

customers_df = grocery_df.groupby("customer_id").agg({"customer_id": "count"})
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   25415089|            27|
|   18869981|             2|
|   51052113|            22|
|   13203460|             3|
|   30238476|             1|
|   33212984|             1|
|   37757447|            13|
|   51706294|             3|
|   51506042|             1|
|   31325270|             1|
|   21174138|             9|
|   44662810|             1|
|   33282498|             1|
|   22050232|             1|
|   22279960|             1|
|   46169082|             1|
|   48542530|             2|
|   52857540|             2|
|   46727040|             7|
|   42665845|             1|
+-----------+--------------+
only showing top 20 rows



In [32]:
# Check datatypes
customers_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_count: long (nullable = false)



In [33]:
# Change datatypes to match schema

customers_df = customers_df \
  .withColumn("customer_id" ,
              customers_df["customer_id"]
              .cast(IntegerType()))   \
  .withColumn("customer_count",
              customers_df["customer_count"]
              .cast(IntegerType()))
  
# Recheck datatypes
customers_df.printSchema()  


root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



## Create the "vine_table".

In [34]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = grocery_df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine')
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R26MV8D0KG6QI6|          5|            0|          0|   N|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|
| RIG9AWFOGRDVO|          2|            1|          1|   N|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|
| RB15NBVY5ELVW|          5|            2|          2|   N|
| R56358YM1ZJ7I|          5|            0|          0|   N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|
|R155ZMVLD5C9BP|          4|            3|          4|   N|
|R2IXW43PUNYM0G|          4|            1|          1|   N|
|R3LYEG1QCK2BG0|          5|            0|          0|   N|
|R20LREICPM3YH0|          5|            

In [35]:
# Check datatypes
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)



In [36]:
# Change datatypes to match schema
vine_df = vine_df \
  .withColumn("star_rating" ,
              vine_df["star_rating"]
              .cast(IntegerType()))   \
  .withColumn("helpful_votes",
              vine_df["helpful_votes"]
              .cast(IntegerType()))    \
  .withColumn("total_votes"  ,
              vine_df["total_votes"]
              .cast(IntegerType()))

# Recheck datatypes
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)